Stage 3: Python  notebook: Demonstrate simple queries in Python notebooks over your data. In particular, apply sentiment analysis to data in different dates and periods and generate simple plots. Enable search queries over the data.

In [ ]:
import lithops

analysis_folder_name = ''

# Retrieve analysis_folder from Stage 1 notebook
%store -r analysis_folder_name

if len(analysis_folder_name) == 0:
    print('You have not defined any folder yet (use Stage 1 notebook or the \'analysis_folder\' variable directly)')
else:
    bucket_name = lithops.Storage().bucket
    data = lithops.Storage().get_object(bucket_name, analysis_folder_name+'.csv', stream=True)

In [ ]:
import numpy as np
import pandas as pd

pd.set_option("display.precision", 2)
df = pd.read_csv(data, parse_dates=['date'])
df['date_norm'] = df['date'].dt.normalize()

In [ ]:
import calplot

# # of tweets
calplot.calplot(df['date_norm'].value_counts(), cmap='YlGn', colorbar=True)

In [ ]:
# sentiment
calplot.calplot(df.groupby('date_norm')['sentiment'].mean(), cmap='YlGn', colorbar=True)

In [ ]:
import plotly.graph_objects as go

import plotly.offline

grouped = df.groupby(pd.Grouper(key='date', freq='Q'))['sentiment'].value_counts()

x_axis = grouped.index.levels[0]
fig = go.Figure()

for n_stars in grouped.index.levels[1]:
    arr = []
    for day in grouped.index.levels[0]:
        d = grouped.get(day)
        if d is not None:
            arr.append(d.get(n_stars, default=0))
    fig.add_trace(go.Bar(x=x_axis, y=arr, name=n_stars))

fig.update_layout(barmode='stack')
#fig.update_yaxes(type="log")
fig.show()
plotly.offline.plot(fig, filename = 'results/SentimentChart1.html', auto_open=False)

In [ ]:
import plotly.express as px
groupLangs = df.groupby('lang')

fig = px.bar(groupLangs, x=0, y=groupLangs.size(), title="Language usage")
fig.update_layout(xaxis_title="Language", yaxis_title="Number of tweets")
fig.update_yaxes(type="log")
fig.show()

In [ ]:
import plotly.figure_factory as ff

# Confusion matrix
conf_matrix = pd.crosstab(df['guessedLang'], df['lang'], rownames=['Our guess'], colnames=['Twitter'])

fig = ff.create_annotated_heatmap(conf_matrix.to_numpy(), x=conf_matrix.columns.array, y=conf_matrix.index.array, showscale=True)
fig.update_xaxes(title_text='Twitter')
fig.update_yaxes(title_text='Our guess')
fig.show()

In [ ]:
df.describe()

In [ ]:
df.info()
#df.memory_usage()

In [ ]:
import geopandas
# convert pandas dataframe to GeoPandas dataframe
aux = df[df["coordinates"].notna()]
aux["coordinates"] = aux["coordinates"].apply(lambda c: "POINT (" + c[c.find("=")+1:c.find(",")]+ " " + c[c.find("=", c.find("=")+1)+1:])

aux['coordinates'] = geopandas.GeoSeries.from_wkt(aux['coordinates'])
gdf = geopandas.GeoDataFrame(aux, geometry='coordinates')

# load world map
world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))


In [ ]:
ax = world.plot(color='white', edgecolor='black', figsize=(12,8))

gdf.plot(ax=ax)

In [ ]:
import folium
from folium import plugins

map = folium.Map(location = [15,30], tiles='Cartodb dark_matter', zoom_start = 2)

heat_data = [[point.xy[1][0], point.xy[0][0]] for point in gdf.geometry ]

heat_data
plugins.HeatMap(heat_data).add_to(map)

map

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(20,8))
world.plot(ax=ax, alpha=0.4, color='grey')
gdf.plot(column='sentiment', ax=ax, legend=True)

In [ ]:
# The dataframe_sql library lets you query pandas DataFrames
# using SQL syntax. Unlike other libraries, dataframe_sql
# performs native pandas operations instead of using a
# database like SQLite as backend.
# 
# Learn more at: https://github.com/zbrookle/dataframe_sql

from dataframe_sql import register_temp_table, query

# We will use the table name "dataframe"
# to refer to our dataframe
register_temp_table(df, "dataframe")

In [ ]:
# Query top 10 tweets written in Catalan with the most retweets
query("""SELECT user,content,retweetCount
             FROM dataframe
             WHERE lang = 'ca'
             ORDER BY retweetCount DESC
             LIMIT 10""")

In [ ]:
# Count all tweets written in Italian
query("""SELECT COUNT(*) AS tweets_it
             FROM dataframe
             WHERE lang = 'it'""")

In [ ]:
# Get the top 10 tweets with the most likes
query("""SELECT *
             FROM dataframe
             ORDER BY likeCount DESC
             LIMIT 10""")

In [ ]:
# Get the overall sentiment of tweets by the user "EU_H2020"
query("""SELECT AVG(sentiment) AS avg_sentiment
             FROM dataframe
             WHERE user = 'EU_H2020'""")

In [ ]:
# Get tweets by @cloudbutton2020
query("""SELECT *
             FROM dataframe
             WHERE user = 'cloudbutton2020'""")